In [1]:
import numpy as np
import igraph as ig
import dionysus as d
import multiprocessing
# from dask.distributed import Client
from zigzag import sliding_windows, zigzag_network
from wrcf import wrcf_diagram
from sliced_wasserstein import diagram_array, SW_approx
import dill

In [12]:
def random_edge_presences(T, f):
    """Generate random times sampled over a periodic distribution.
    :param T: time range
    :param f: frequency
    :return: an array of times.
    """
    density = np.sin(f * np.arange(T)) + 1
    density /= np.sum(density)
    samplesize = np.random.randint(T//2)
    times = np.random.choice(np.arange(T), size=samplesize, replace=False, p=density)
    times = np.sort(times)
    return times, density, samplesize

In [17]:
a = random_edge_presences(200, 15/200)

In [18]:
a[-1]

48

In [19]:
a[0]

array([  0,   1,   3,   5,  11,  14,  15,  16,  20,  22,  25,  27,  29,
        31,  39,  43,  50,  72,  87,  92,  93,  95,  98,  99, 101, 104,
       111, 112, 114, 115, 116, 118, 120, 130, 160, 161, 172, 176, 177,
       182, 183, 184, 185, 187, 190, 193, 196, 198])

In [3]:
def remove_inf(dgm):
    """Remove infinite points in a persistence diagram.
    :param dgm: Diagram
    :return: the same diagram without the infinite points.
    """
    res = d.Diagram()
    for p in dgm:
        if p.death != np.inf:
            res.append(p)
    return res

In [4]:
## Global parameters
NODES = 40
EDGE_PROB = 0.9
TIME_RANGE = 200
FREQ = 15/TIME_RANGE
N_WINDOWS = 20
## Computations
ZIGZAG_PERS = True
WRCF_PERS = True
SW_KERNEL = True
BOTTLENECK_DIST = True

In [5]:
basegraph = ig.Graph.Erdos_Renyi(NODES, EDGE_PROB)
g = ig.Graph()
g.add_vertices(len(basegraph.vs))

for e in basegraph.es:
    times = random_edge_presences(TIME_RANGE, FREQ)
    for t in times:
        g.add_edge(e.source, e.target, time=t)
print("done.")

done.


In [21]:
sum(g.get_edge_dataframe()['time'].unique())

19270